In [1]:
from utils import load_data
import numpy as np
import models 

In [2]:
data_file = 'D:/zhuomian/file/弱网中鲁棒性的恶意节点检测/exp/dataset/20241106_testTX_constantUniform_simtime-100_num_nodes-50_BHradio-0_SFradio-0_fieldLength-20_uniform-1/ProcessedMonitorSnifferRx.csv'

In [3]:
# np.set_printoptions(threshold=np.inf)  # 这将打印所有内容，去掉折叠
processed_data, adj, T, group_num = load_data(1,data_file)

[tensor([  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          8.0000,  47.2674, -45.6511, -92.9186]), tensor([  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          3.0000,  24.5063, -68.2015, -92.7078]), tensor([  0.0000,   4.0000,   2.0000,   0.0000,   8.0000,   0.0000,   0.0000,
          2.0000,  19.6734, -68.2015, -87.8749]), tensor([  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          2.0000,  25.3501, -68.2015, -93.5517]), tensor([  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          2.0000,  22.8176, -68.2015, -91.0191]), tensor([  0.0000,   3.0000,   3.0000,   0.0000,   7.0000,   0.0000,   0.0000,
          2.0000,  21.9685, -68.2015, -90.1700]), tensor([  0.0000,   4.0000,   3.0000,   0.0000,  11.0000,   0.0000,   0.0000,
          6.0000,  89.1317,  -0.5504, -89.6821]), tensor([  0.0000,   3.0000,   3.0000,   0.0000,   9.0000,   0.0000,   0.0000,
          2.0000,  22.068

D:\zhuomian\file\弱网中鲁棒性的恶意节点检测\exp\utils.py:263: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  np_hypernode_features = np.array(hypernode_features)  # 转换为 NumPy 数组


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (484,) + inhomogeneous part.

In [ ]:
# 设置numpy的打印选项，显示更多行列


# 打印节点特征（processed_data.x）
# print(T)
# print(adj[-1])

In [ ]:
# 打印边连接关系（processed_data.edge_index）
print(processed_data[-1].edge_index.numpy().size)

In [ ]:
# 打印节点标签（processed_data.y）
# print(processed_data[0].y.numpy())
print(processed_data[-1].x[:,0:8].numpy().size)

In [ ]:
train_model = models.HypergraphModel(8, 3)
print(train_model)

In [ ]:
import time
import torch.optim as optim
from utils import accuracy
import torch.nn.functional as F
import torch
criteria = torch.nn.CrossEntropyLoss()
learning_rate = 0.001
weight_decay = 5e-4
optimizer = optim.Adam(train_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
acc_measure = accuracy
def train(epoch):
    t = time.time()
    train_model.train()
    
    
    optimizer.zero_grad()
    # print(processed_data[0].x[0:8].shape)
    # print(processed_data[0].edge_weights.shape)
    # print(processed_data[0].edge_index.shape)
    for i in range(group_num):
        print(i)
        output = train_model(processed_data[1].x[:, 0:8], processed_data[1].edge_index, 
                                 processed_data[1].edge_weights, processed_data[1].x[:, 8:11], adj[1], T[1])
        # print(output)
        labels = processed_data[1].y
        loss_train = criteria(output, labels)

        acc_train = acc_measure(output, labels)
        
        loss_train.backward(retain_graph=True)
        optimizer.step()
        
        train_model.eval()
        output = train_model(processed_data[1].x[:, 0:8], processed_data[1].edge_index, 
                                 processed_data[1].edge_weights, processed_data[1].x[:, 8:11], adj[1], T[1])
        # print(output)
        loss_val = criteria(output, labels)
        acc_val = acc_measure(output, labels)
        print(
              'loss_train: {:.4f}'.format(loss_train.item()),
              'acc_train: {:.4f}'.format(acc_train.item()),
              'loss_val: {:.4f}'.format(loss_val.item()),
              'acc_val: {:.4f}'.format(acc_val.item()),
              'time: {:.4f}s'.format(time.time() - t))

In [ ]:
def test():
    train_model.eval()
    output = train_model(processed_data[1].x[:, 0:8], processed_data[1].edge_index, 
                             processed_data[1].edge_weights, processed_data[1].x[:, 8:11], adj[1], T[1])
    labels = processed_data[0].y
    loss_test = criteria(output, labels)
    acc_test = acc_measure(labels, output)
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))
    return acc_test.item()

In [ ]:
torch.autograd.set_detect_anomaly(True)
num_epochs = 100
early_stopping = 30
val_watch = []
for epoch in range(num_epochs):
    val_watch.append(train(epoch))
    test()
    if epoch > early_stopping and val_watch[-1] > np.mean(val_watch[-(early_stopping + 1):-1]):
        print("Early stopping...")
        break
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
print("Printing the weights : ")